nota: si se tiene problemas para conectar a neo4j desde el navegador probar
* host: bolt://localhost:7687
* usuario: neo4j
* password: neo4j

1) si funciona, pedira cambiar el pass: poner cualquier cosa y usar como comando en el navegador
    * :server change-password
    * despues volver al password neo4j
   
2) si hay probmemas, usar desde la consola donde esta el docker:
    * docker-compose exec neo4j bash
    * neo4j-admin set-initial-password neo4j

    y reintentar lo anterior

librerias requeridas:
* pip install py2neo

In [7]:
from py2neo import Graph, Node, Relationship, NodeMatcher

# conectarse
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4j"))

# graph.delete_all()

crear nodos

In [2]:
# nicole = Node("Person", name="Nicole", age=24)
# drew = Node("Person", name="Drew", age=20)

# mtdew = Node("Drink", name="Mountain Dew", calories=9000)
# cokezero = Node("Drink", name="Coke Zero", calories=0)

# coke = Node("Manufacturer", name="Coca Cola")
# pepsi = Node("Manufacturer", name="Pepsi")

# graph.create(nicole | drew | mtdew | cokezero | coke | pepsi)

crear relaciones

In [3]:
# graph.create(Relationship(nicole, "LIKES", cokezero))
# graph.create(Relationship(nicole, "LIKES", mtdew))
# graph.create(Relationship(drew, "LIKES", mtdew))
# graph.create(Relationship(coke, "MAKES", cokezero))
# graph.create(Relationship(pepsi, "MAKES", mtdew))

obtener data (por una query con parametros)

In [4]:
# query = """
# MATCH (person:Person)-[:LIKES]->(drink:Drink)
# WHERE person.name = {nicole_name}
# RETURN person.name AS name, drink.name AS drink
# """

# data = graph.run(query, nicole_name='Nicole')

# for d in data:
#     print(d)

In [8]:
import json
f = open('Santiago_de_Chile.json', 'r')
metros = json.loads(f.read())
f.close()

matcher = NodeMatcher(graph)

for metro in metros:
    current_station_node = matcher.match("Station", name=metro['station']).first()
    if current_station_node:
        if current_station_node['line'] != metro['line']:
            current_station_node['line'] = '{},{}'.format(current_station_node['line'], metro['line'])
            graph.push(current_station_node)
    else:
        current_station_node = Node("Station", name=metro['station'], line=metro['line'])
        graph.create(current_station_node)

    if metro['previous'] != None:
        previous_node = matcher.match("Station", name=metro['previous']['station']).first()
        graph.create(Relationship(previous_node, "CONNECT", current_station_node, distance=metro['previous']['distance_in_meters']))
    if metro['next'] != None:
        next_node = matcher.match("Station", name=metro['next']['station']).first()
        if next_node:
            graph.create(Relationship(current_station_node, "CONNECT", next_node, distance=metro['next']['distance_in_meters']))

* Seleccione y muestre todas las estaciones correspondientes a una lı́nea de metro (Excluyendo lı́nea 4A).

In [9]:
query = """
MATCH (n)
WHERE n.line contains {required_line} AND (not(n.line contains 'L4A'))
RETURN n
"""

data = graph.run(query, required_line='L4')

for d in data:
    print(d)

<Record n=(_19:Station {line: 'L1,L4', name: 'Tobalaba'})>
<Record n=(_88:Station {line: 'L3,L4', name: 'Plaza Ega\u00f1a'})>
<Record n=(_90:Station {line: 'L4', name: 'Crist\u00f3bal Col\u00f3n'})>
<Record n=(_91:Station {line: 'L4', name: 'Francisco Bilbao'})>
<Record n=(_92:Station {line: 'L4', name: 'Pr\u00edncipe de Gales'})>
<Record n=(_93:Station {line: 'L4', name: 'Sim\u00f3n Bol\u00edvar'})>
<Record n=(_94:Station {line: 'L4', name: 'Los Orientales'})>
<Record n=(_95:Station {line: 'L4', name: 'Grecia'})>
<Record n=(_96:Station {line: 'L4', name: 'Los Presidentes'})>
<Record n=(_97:Station {line: 'L4', name: 'Quil\u00edn'})>
<Record n=(_98:Station {line: 'L4', name: 'Las Torres'})>
<Record n=(_99:Station {line: 'L4', name: 'Macul'})>
<Record n=(_101:Station {line: 'L4,L5', name: 'Vicente Vald\u00e9s'})>
<Record n=(_102:Station {line: 'L4', name: 'Rojas Magallanes'})>
<Record n=(_103:Station {line: 'L4', name: 'Trinidad'})>
<Record n=(_104:Station {line: 'L4', name: 'San Jos\u0

* Muestre las 10 estaciones mas cercanas entre si

In [11]:
query = """
MATCH(first)-[c:CONNECT]->(second:Station)
RETURN first.name, c.distance, second.name 
ORDER BY c.distance 
LIMIT 10
"""

data = graph.run(query)

for d in data:
    print(d)

<Record first.name='La Moneda' c.distance=400 second.name='Universidad de Chile'>
<Record first.name='Los Héroes' c.distance=460 second.name='La Moneda'>
<Record first.name='Universidad de Chile' c.distance=480 second.name='Santa Lucía'>
<Record first.name='Parque Bustamante' c.distance=500 second.name='Santa Isabel'>
<Record first.name='Las Rejas' c.distance=520 second.name='Ecuador'>
<Record first.name='Santa Lucía' c.distance=530 second.name='Universidad Católica'>
<Record first.name='San Alberto Hurtado' c.distance=540 second.name='Universidad de Santiago'>
<Record first.name='Carlos Valdovinos' c.distance=540 second.name='Camino Agrícola'>
<Record first.name='Pedro de Valdivia' c.distance=550 second.name='Los Leones'>
<Record first.name='Unión Latinoamericana' c.distance=560 second.name='República'>
